# Tutorial 3

We use the same example as tutorial 2, but with an internally generated trajectory.

In [ ]:
%matplotlib inline
import numpy as np

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; } </style>"))

We do not use an input channel, but we create the trajectory directly inside eTaSL.  This can be very handy, because a trajectory generation component does not need to be synchronized with the eTaSL component.  Another advantage is that the derivative of the trajectory does not need to be manually computed.
For example, the input channels can be used to communicate some parameters of the motion (such as end pose, max. velocity, etc...).  Since trajectories needs to start from the current pose, the following eTaSL function can be useful:

    initial_value( time,  expr)
        
            returns the value of expr that occurred when time was 0. (in other words, 
            the expression is evaluated once when time <=0; after that the value is fixed.

In the case here, the two frequencies for the Lissajous figure are communicated from outside the eTaSL component.  This example does not use `initial_value`

In [ ]:
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
e = etasl_simulator(regularization_factor= 0.0000001)

e.readTaskSpecificationString("""
    require("context")
    require("geometric")
    local u=UrdfExpr();
    local fn = rospack_find("etasl_py_examples").."/robots/ur10_robot.urdf"
    u:readFromFile(fn)
    u:addTransform("ee","tool0","base_link")
    local r = u:getExpressions(ctx)
    ee = r.ee
    robot_jname={"shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"}
    robot_jval = {}
    for i=1,#robot_jname do
       robot_jval[i]   = ctx:getScalarExpr(robot_jname[i])
    end
""")

e.readTaskSpecificationString(""" 

    f1 = ctx:createInputChannelScalar("f1",0)
    f2 = ctx:createInputChannelScalar("f2",0)

    tgt_x = sin(f1*time)*0.15 + 0.7
    tgt_y = sin(f2*time)*0.1 +0.4
    tgt_z = 0

    d = Variable{context=ctx, name="d", vartype="feature"}


    Constraint{
        context=ctx,
        name="laserdistance",
        expr=d,
        target_lower = 0.3,
        target_upper = 0.45,
        K = 4
    }


    laserspot = ee*vector(0,0,d)

    Constraint{
        context=ctx,
        name="x",
        expr = tgt_x - coord_x(laserspot),
        priority = 2,
        K        = 4
    }
    Constraint{
        context=ctx,
        name="y",
        expr = tgt_y - coord_y(laserspot),
        priority = 2,
        K        = 4
    }
    Constraint{
        context=ctx,
        name="z",
        expr = tgt_z - coord_z(laserspot),
        priority = 2,
        K        = 4
    }

    ctx:setOutputExpression("error_x",coord_x(laserspot)-tgt_x)
    ctx:setOutputExpression("error_y",coord_y(laserspot)-tgt_y)
    ctx:setOutputExpression("error_z",coord_z(laserspot)-tgt_z)
    ctx:setOutputExpression("laser_x",coord_x(laserspot))
    ctx:setOutputExpression("laser_y",coord_y(laserspot))
    ctx:setOutputExpression("laser_z",coord_z(laserspot))
    
""")

The simulator class can also handle input that is not given as a matrix with a row for each time step.
It can also handle a list input of input variables that are set once.


In [ ]:

pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint','d']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,-2.0,-np.pi*0.5,0.1,0.1])

N=4000
dt=0.005

time = np.arange(0,N)*dt
inp_lbl=['f1','f2']
inp=[1, 2.5]
e.setInputTable(inp_lbl,inp)

e.initialize(initial_jpos, pos_lbl)
e.simulate(N=N,dt=dt)

# Plotting (using the Bokeh library to provide interactive plots)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from etasl_py.bokehplots import plotv, append_units

output_notebook()
print("positions : ")
plotv(e.TIME,e.POS, append_units(e.POS_LBL,[" [rad]"]*6 + [" [m]"]) )
print("velocities : ")
plotv(e.TIME,e.VEL, append_units(e.POS_LBL,[" [rad/s]"]*6 + [" [m/s]"]))
print("control errors : ")
plotv(e.TIME,e.OUTP[:,0:3],append_units(e.OUTP_LBL[0:3]," [m]"))

In [ ]:
f = figure(x_axis_label='x[m]', 
                y_axis_label= 'y[m]',plot_width=800, plot_height=400 )
#f.line(e.INP[:,0],e.INP[:,1], line_width=2,color='red')
f.line(e.OUTP[:,3],e.OUTP[:,4], line_width=1)
show(f)